## Decision Tree Classifier

In [ ]:
import numpy as np

In [ ]:
data = np.array([
    [12.0, 1.5, 1, 'Wine'],
    [5.0, 2.0, 0, 'Beer'],
    [40.0, 0.0, 1, 'Whiskey'],
    [13.5, 1.2, 1, 'Wine'],
    [4.5, 1.8, 0, 'Beer'],
    [38.0, 0.1, 1, 'Whiskey'],
    [11.5, 1.7, 1, 'Wine'],
    [5.5, 2.3, 0, 'Beer']
]
)

### Preprocessing

We map the class labels to integers and separate the data into features and target values.


In [ ]:
for row in data:
    if row[3] == 'Wine':
        row[3] = 0
    elif row[3] == 'Beer':
        row[3] = 1
    elif row[3] == 'Whiskey':
        row[3] = 2
    else:
        row[3] = None

# Just for naming things nicely later
features = ['Alcohol Content', 'Sugar', 'Color']
classes = ['Wine', 'Beer', 'Whiskey']

In [ ]:
# Separating features (X) and labels (y)
X_train = np.array(data[:, :3], dtype = float)
y_train = np.array(data[:, 3], dtype = int)

### Impurity Functions

Both Gini impurity and entropy functions are implemented here.


In [ ]:
def giniImpurity(y):
    m = len(y)
    if m == 0:
        return 0  # No data, no impurity
    counts = np.bincount(y)  # Count how many of each class
    prob = counts / m
    return 1 - np.sum(prob ** 2)  # Standard Gini formula

In [ ]:
def entropy(y):
    m = len(y)
    if m==0:
        return 0
    counts = np.bincount(y)
    prob = counts/m
    prob = prob[prob>0] # Remove 0 probability to avoid log0
    return -np.sum(prob*np.log2(prob))

### Tree Node

In [ ]:
class Node():
    def __init__(self, featureIndex, threshold, left, right, value):
        self.featureIndex = featureIndex  # which feature to split on
        self.threshold = threshold        # value to split at
        self.left = left                  # left subtree
        self.right = right                # right subtree
        self.value = value                # final predicted class (if leaf)


### Finding the best split

Have implemented functions to find the best split features and thresholds using both Gini impurity and entropy

In [ ]:
# Tries all possible splits and picks the one with lowest Gini impurity
def findBestSplitGini(X, y):

    numSamples, numFeatures = X.shape
    minGiniImp = float('inf')  # start with something huge
    bestFeature = None
    bestThreshold = None

    for i in range(numFeatures):  # for each feature
        thresholds = np.unique(X[:, i])  # try each unique value as a split point
        for threshold in thresholds:
            left = X[:, i] < threshold
            right = X[:, i] >= threshold

            # weighted average of Gini impurities on both sides
            totalGiniImp = ((len(left)/numSamples)*giniImpurity(y[left])) + ((len(right)/numSamples)*giniImpurity(y[right]))
            if totalGiniImp < minGiniImp:
                minGiniImp = totalGiniImp
                bestFeature = i
                bestThreshold = threshold

    return bestFeature, bestThreshold

In [ ]:
# Same as above but using Entropy in place of Gini
def findBestSplitEntropy(X, y):
    numSamples, numFeatures = X.shape
    minEnt = float('inf')
    bestFeature = None
    bestThreshold = None
    for i in range(numFeatures):
        thresholds = np.array(np.unique(X[:,i]))
        for threshold in thresholds:
            left = np.array(X[:,i]<threshold)
            right = np.array(X[:,i]>=threshold)
            totalEnt = ((len(left)/numSamples)*entropy(y[left]))+((len(right)/numSamples)*entropy(y[right]))
            if totalEnt<=minEnt:
                minEnt = totalEnt
                bestFeature = i
                bestThreshold = threshold
    return bestFeature, bestThreshold


### Fitting training data to make tree

In [ ]:
# Recursive function to build the decision tree
def makeTree(X, y, maxDepth, curDepth=0, minSamples=1):
    # Base case: if tree is deep enough, or data is pure (only one class), or no data left
    if curDepth == maxDepth or len(np.unique(y)) == 1 or len(y) <= 0:
        return Node(None, None, None, None, np.argmax(np.bincount(y)))  # return majority class

    # Otherwise, find best feature + threshold to split on
    bestFeature, bestThreshold = findBestSplitEntropy(X, y)

    if bestFeature is None:  # just in case we can't split
        return Node(None, None, None, None, np.argmax(np.bincount(y)))

    # Actually split the data
    left = X[:, bestFeature] < bestThreshold
    right = X[:, bestFeature] >= bestThreshold

    # Recursively build left and right subtrees
    leftTree = makeTree(X[left], y[left], maxDepth, curDepth+1)
    rightTree = makeTree(X[right], y[right], maxDepth, curDepth+1)

    # Return node with references to the two subtrees
    return Node(bestFeature, bestThreshold, leftTree, rightTree, None)


### Inference

In [ ]:
def predict(tree, x):
    if tree.value is not None:
        return tree.value
    if x[tree.featureIndex] < tree.threshold:
        return predict(tree.left, x)
    else:
        return predict(tree.right, x)

def predictMultiple(tree, X):
    return np.array([predict(tree, x) for x in X])

### Testing the tree

In [ ]:
test_data = np.array([
    [6.0, 2.1, 0],   # Expected: Beer
    [39.0, 0.05, 1], # Expected: Whiskey
    [13.0, 1.3, 1]   # Expected: Wine
])

In [ ]:
# Build tree and make predictions
tree = makeTree(X_train, y_train, 3, 0)
predictions = predictMultiple(tree, test_data)

# Print out the predictions in string form
for prediction in predictions:
    print(classes[prediction])

Beer
Whiskey
Wine


### Printing tree in a formatted way

In [ ]:
# Recursively prints tree in a formatted manner
def printFormattedTree(node):
    if node.value is not None:
        print("the class is", classes[node.value])
    else:
        print("if the feature", features[node.featureIndex], "<", node.threshold, ": go left and", end=" ")
        printFormattedTree(node.left)
        print("if the feature", features[node.featureIndex], ">=", node.threshold, ": go right and", end=" ")
        printFormattedTree(node.right)

printFormattedTree(tree)

if the feature Color < 1.0 : go left and the class is Beer
if the feature Color >= 1.0 : go right and if the feature Sugar < 1.2 : go left and the class is Whiskey
if the feature Sugar >= 1.2 : go right and the class is Wine
